In [1]:
import GetPropertiesAPI as GP
from collections import Counter
import pandas as pd
import importlib
importlib.reload(GP)
import statistics as s
import operator
from collections import defaultdict
import csv 

In [2]:
aid_list = []
gidAidDict = {}
for gid in range(1,1000):
    aid = GP.getAnnotID(gid)
    aid_list.append(aid)
    gidAidDict[gid] = aid
    
aid_list = list(filter(lambda x: x != None,aid_list))

In [3]:
aids = []
for aid_l in aid_list:
    for aid in aid_l:
        aids.append(aid)

In [4]:
aidContribTupList = []
for aid in aids:
    contrib = GP.getImageFeature(aid,'image_contributor_tag')
    aidContribTupList.append((aid,contrib[0]))

In [5]:
aidNidMap = {}
aidNamesMap = {}
aidNidTupList = [] # modified

for aid in aids:
    nid = GP.getImageFeature(aid,'nids')
    aidNidMap[aid] = nid
    aidNidTupList.append((aid,nid[0]))

In [6]:
nids = []
for aid in aidNidMap.keys():
    nids.append(aidNidMap[aid][0])

In [7]:
nids = list(filter(lambda x : x > 0,nids))

In [8]:
counter_nid= Counter(nids)


In [9]:
gidAidDict
aidGidTupList = [] # key : aid and value : gid # modified
for gid in gidAidDict.keys():
    if gidAidDict[gid] != None:
        for aid in gidAidDict[gid]:
            aidGidTupList.append((aid,gid))

In [10]:
aidGidDf = pd.DataFrame(aidGidTupList,columns = ['AID','GID']) 
aidNidDf = pd.DataFrame(aidNidTupList,columns = ['AID','NID']) 
aidContribDf = pd.DataFrame(aidContribTupList,columns = ['AID','CONTRIBUTOR'])
aidNidDf = aidNidDf[(aidNidDf['NID']>0)]

In [11]:
aidGidNidDf = pd.merge(aidGidDf,aidNidDf,left_on = 'AID',right_on = 'AID')
aidGidNidContribDf = pd.merge(aidGidNidDf,aidContribDf,left_on = 'AID',right_on = 'AID')

In [12]:
aidGidNidContribDf.to_csv('../FinalResults/IndividualAnimalPhotoCount.csv',index=False)

In [14]:
with open('../FinalResults/IndividualAnimalPhotoCount.csv') as f: # read from csv file into a key : GID and value : CONTRIBUTOR
    reader = csv.DictReader(f)
    gidContribMap = { line['GID']: line['CONTRIBUTOR'] for line in reader }
        
len(gidContribMap)

426

In [15]:
ContribTotal = {} # dict with key : CONTRIBUTOR and value: Total photos taken

for gid,contrib in gidContribMap.items():
     ContribTotal[contrib] = ContribTotal.get(contrib,0) + 1

print(s.mean(ContribTotal.values()))
print(s.stdev(ContribTotal.values()))

42.6
42.836121828817944


In [17]:
with open('../FinalResults/IndividualAnimalPhotoCount.csv') as f2: # read from csv file into a Dict with key : AID and value : GID, NID, CONTRIBUTOR
    reader2 = csv.DictReader(f2)
    aidToGidNidContribMap = { line['AID']: [line['GID'], line['NID'], line['CONTRIBUTOR']] for line in reader2 }

In [18]:
NidContribTotal = {} # dict with key : NID, CONTRIBUTOR and value: Total photos taken
for aid,(gid,nid,contrib) in aidToGidNidContribMap.items():
     NidContribTotal[nid,contrib] = NidContribTotal.get((nid,contrib),0) + 1

In [19]:
csv_out = csv.writer(open('nidtoContributor.csv', 'w')) # rename file, results of Nid to Contributor to Total
csv_out.writerow(['NID', 'CONTRIBUTOR', 'TOTAL'])
for (Nid, Contrib), value in NidContribTotal.items():
    csv_out.writerow([Nid, Contrib, value])

In [20]:
#from collections import defaultdict

averageCountofPictures = defaultdict(list) # dict where key : NID and values: list of pictures taken per photographer

for (nid, contrib), total in NidContribTotal.items(): 
    averageCountofPictures[nid].append(total)
       
#averageCountofPictures

In [21]:
countUniquePhotoPerPic = {} # dict where key : NID and values : # of CONTRIBUTERS

for (nid, contrib), total in NidContribTotal.items():
    countUniquePhotoPerPic[nid] = countUniquePhotoPerPic.get((nid),0) + 1 

#countUniquePhotoPerPic['741']

In [84]:
#aidToSexMap={} #dict where key : AID and values : sex

#for aid in aids:
#    aidToSexMap[aid]=GP.getImageFeature(aid, 'sex_texts') 

In [76]:
#ContribToNidMap = defaultdict(list) # dict where key : Contributers and values : List of NID ***Problem is duplicates

#for aid,(gid,nid,contrib) in aidToGidNidContribMap.items(): #aidToGidNidContribMap is a dict. straight from excel sheet
#    ContribToNidMap[contrib].append(nid)

In [67]:
#with open('results.csv') as f4: #get info from excel sheet, dict with key : AID and values: NID
#    reader4 = csv.DictReader(f4)
#    aidNidDict = { line['AID']: line['NID'] for line in reader4 }

In [68]:
#nidToAidMap = {} #dict where key : Nid and values : List of AID
#for aid, nid in aidNidDict.items():
#    nidToAidMap[nid] = nidToAidMap.get(nid, [])
#    nidToAidMap[nid].append(aid)

In [ ]:
#JUST LOOK FROM HERE

In [46]:
contribToSexMap = defaultdict(list) # dict where key : contributor and values : List of sex 

for aid,(gid,nid,contrib) in aidToGidNidContribMap.items():
    contribToSexMap[contrib].append(GP.getImageFeature(aid, 'sex_texts')[0])

In [47]:
contribAnimSexCount = {}
for key in contribToSexMap.keys():
    contribAnimSexCount[key] = dict(Counter(contribToSexMap[key]))

In [48]:
contribAnimSexCount

{"NNP GZC Car '10WHITE', Person 'A'": {'Female': 9,
  'Male': 5,
  'UNKNOWN SEX': 2},
 "NNP GZC Car '11WHITE', Person 'A'": {'Female': 89,
  'Male': 34,
  'UNKNOWN SEX': 16},
 "NNP GZC Car '12WHITE', Person 'A'": {'Female': 11,
  'Male': 8,
  'UNKNOWN SEX': 12},
 "NNP GZC Car '13WHITE', Person 'A'": {'Female': 10,
  'Male': 10,
  'UNKNOWN SEX': 3},
 "NNP GZC Car '13WHITE', Person 'B'": {'Female': 25,
  'Male': 31,
  'UNKNOWN SEX': 9},
 "NNP GZC Car '14WHITE', Person 'A'": {'Female': 64,
  'Male': 35,
  'UNKNOWN SEX': 11},
 "NNP GZC Car '15WHITE', Person 'A'": {'Female': 32,
  'Male': 30,
  'UNKNOWN SEX': 9},
 "NNP GZC Car '15WHITE', Person 'B'": {'Female': 8, 'Male': 1},
 "NNP GZC Car '15WHITE', Person 'C'": {'Female': 2,
  'Male': 2,
  'UNKNOWN SEX': 2},
 "NNP GZC Car '15WHITE', Person 'D'": {'UNKNOWN SEX': 1}}

In [152]:
contribToSexMap2=defaultdict(list) # dict where single list for values

for contrib,sex in contribToSexMap.items():
    contribToSexMap2[contrib].append(sum(sex, []))
        
            
contribToSexMap2

defaultdict(list,
            {'GIRM_MUGU_20,hyrule:joncrall:/media/raid/work/GIRM_MUGU_20,,,,,0': [['Male',
               'Female',
               'Male',
               'Male',
               'Male',
               'UNKNOWN SEX',
               'Female',
               'UNKNOWN SEX',
               'Female',
               'Male',
               'Male',
               'Female',
               'Male',
               'Male',
               'UNKNOWN SEX',
               'Male',
               'Male',
               'Female',
               'Male',
               'Female',
               'Female',
               'Male',
               'Female',
               'Male',
               'UNKNOWN SEX',
               'Male',
               'Male',
               'Female',
               'Female',
               'Male',
               'Male',
               'Female',
               'Male',
               'Male',
               'Female',
               'Male',
               'Female',
         

In [43]:
a = GP.getImageFeature('9448','age_months_est')
GP.getAgeFeatureReadableFmt(a)

['infant']